# Performing data analysis first

## Importing necessary libraries

In [ ]:
import pandas as pd                                           #importing pandas library
import matplotlib.pyplot as mpt                    #importing python maths plot library
import seaborn as sb                                         #to work with more efficient statistical graph plots
import datetime as dt                                       #to import pandas timeStamp library
from decimal import Decimal                         #decimal library is used for more accuracy in arithmetic operations

In [ ]:
sb.set()                                                                  #applying default seaborn scaling and themes

## Reading data and storing it in a variable named data

In [ ]:
data = pd.read_csv("../input/911.csv")
data.head()                                                             #default value of head is 5, and it displays top 5 rows of data

## Knowing the dimensions of our dataset by using the shape facility

In [ ]:
data.shape

In [ ]:
data.shape[0]                                        #gives the number of rows

In [ ]:
data.shape[1]                                          #gives the number of columns

## Knowing the information of the columns in the datasets

In [ ]:
data.info()

## Making a list of all the columns

In [ ]:
column_names = list(data.columns)

In [ ]:
column_names

## Making another column having the types of call

In [ ]:
data.title.head()                                         #now we seperate the type of call from the title column

In [ ]:
def call_seperator(x):
    x= x.split(':')                         #here : is the delimeter
    return x[0]                          #this returns the value at 0th index after splitting, which is call type in this case

In [ ]:
data['call_type'] = data['title'].apply(call_seperator)                               #inserting values in another column- call_type

In [ ]:
data.head()

In [ ]:
data['call_type'].nunique()                 #shows the number of unique type of calls 

In [ ]:
data['call_type'].unique()                                      #shows the different values or calls

In [ ]:
data['call_type'].value_counts()                           #counts all the unique value

## We need to convert data to pandas datetime datatype as it does not have one.

In [ ]:
data['timeStamp'] = pd.to_datetime(data['timeStamp'], infer_datetime_format = True)         #infer_datatime_format is used when we want the default format and not our own

In [ ]:
data['timeStamp'].head()

## Getting details of timestamp, with months, year and other things seperately.

In [ ]:
data['year'] = data['timeStamp'].dt.year

In [ ]:
data['month'] = data['timeStamp'].dt.month_name()

In [ ]:
data['day'] = data['timeStamp'].dt.day_name()

In [ ]:
data['hour'] = data['timeStamp'].dt.hour

## Fetching the call details

### Making a function to get the type of emergency from the title column of the dataset

In [ ]:
def type_of_emergency(x):
    x = x.split(':')
    x= x[1]
    return x

In [ ]:
data['emergecy_type'] = data['title'].apply(type_of_emergency)                        #creating a column named emergency_type to store fetched data

In [ ]:
data.head()

## Performing data visualization

In [ ]:
call_types = data['call_type'].value_counts()              #counting different types of calls

In [ ]:
call_types

In [ ]:
mpt.figure(figsize = (12,8))
ax = call_types.plot.bar()
for p in ax.patches:                                                                                                         #patches is used to create whatever type of patch we want for eg. rectangular patch
    ax.annotate(Decimal(str(p.get_height())), (p.get_x(), p.get_height()))       #to annotate all the points on the plot
mpt.xticks(rotation=0)                                                                                                   #this is used to give label on the axis
mpt.savefig("Emergency type vs Frequency.png")

In [ ]:
data.info()                            #fetching info again to visualize in some other way

## Using groupby function to group specific feilds of data.

In [ ]:
calls_data = data.groupby(['month', 'call_type'])['call_type'].count()           #grouped acc. to the types of calls in diff. months

In [ ]:
calls_data.head()

## Calculating percentage of calls

In [ ]:
call_percentage = calls_data.groupby(level = 0).apply(lambda x: round(100*x/float(x.sum())))              #applying lambda funcs to calculate percentage

In [ ]:
call_percentage.head()

### Plotting a graph accordingly

In [ ]:
font = {
    'size': 'x-large',
    'weight': 'bold'
}
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [ ]:
call_percentage = call_percentage.reindex(month_order,  level=0)

In [ ]:
call_percentage = call_percentage.reindex(['EMS', 'Traffic', 'Fire'], level=1)

In [ ]:
call_percentage.head()

In [ ]:
sb.set(rc={'figure.figsize':(12, 8)})
call_percentage.unstack().plot(kind='bar')                                             #making an unstacked graph
mpt.xlabel('Name of the Month', fontdict=font)
mpt.ylabel('Percentage of Calls', fontdict=font)
mpt.xticks(rotation=0)
mpt.title('Calls/Month', fontdict=font)

### Creating a pie chart of the same

In [ ]:
call_percentage = call_percentage.sort_values(ascending=False)

In [ ]:
mpt.figure(figsize=(12,8))
mpt.pie(call_percentage,  labels = call_percentage.index, autopct="%.2f")
mpt.savefig("call percentage pie chart.png")

## Visualizing hourly data

In [ ]:
hours_data = data.groupby(['hour', 'call_type'])['call_type'].count()

In [ ]:
hours_data.head()

## Calculating percentage of hours

In [ ]:
hours_percentage = hours_data.groupby(level=0).apply(lambda x: round(100*x/float(x.sum())))

In [ ]:
hours_percentage.head()

In [ ]:
hours_percentage = hours_percentage.reindex(['EMS', 'Traffic', 'Fire'], level=1)

### Creating a plot accordingly

In [ ]:
sb.set(rc={'figure.figsize':(18, 8)})
hours_percentage.unstack().plot(kind='bar')
mpt.xlabel('Hour of the day', fontdict=font)
mpt.ylabel('Percentage of Calls', fontdict=font)
mpt.xticks(rotation=0)
mpt.title('Calls/Hour', fontdict=font)
mpt.savefig("Percentage of calls vs hours.png")